
# Forum des métiers - Répartition de groupes et allocation de slots

Ce notebook permet d'effectuer des allocations individuelles à des slots de visite de représentants des métiers

Contraintes :

- pas d'élève sans affectation de métier pour un créneau (timeslot) donné,
- le moins possible de métier (job) sans affectation de groupe pour un timeslot donné,
- nombre de souhaits par élève =< timeslots,
- il peut y avoir n représentants pour un métier donné.

Le choix d'affectation utilisé donne la priorité au remplissage de tous les slots des intervenants de manière équilibrée.

La note globale de satisfaction du choix des élèves est un peu plus élevée que sur une stratégie d'affectation selon les préférences élèves pures et le taux d'intervenants sans élève est le plus faible.

La conversion du contenu d'un fichier Excel au formal JSON peut être effectuée sur le site suivant :
https://tableconvert.com/excel-to-json

## Importations des modules

In [190]:
import numpy as np
import pandas as pd
from numpy.random import default_rng
from dataclasses import dataclass

In [191]:
from json import loads
from collections import defaultdict

In [245]:
from modules.typedef import *
from modules.q_utils import *
from modules.student_utils import get_most_interested_student_for_job, get_copy_students, convert_to_student_timetable
from modules.timetable_utils import timetable_student_view, get_timetable_intervenants
from modules.evaluation import get_evaluation

# Allocations des élèves aux timeslots

Un timeslot est une liste des intervenants de métiers pour lesquels la file d'attente 'Q' est renseignée avec les identifiants des élèves qui visiteront cet intervenant pour ce timeslot.

Timeslot :

```Python
[ { 'id' : int , 'batch_size' : int, 'Q' : [ int ] } ]
```

Timetable :

```Python
[ Timeslot ]
```

## Q-intervenant

Pour chaque intervenant considéré tour à tour (et dont le métier ne figure pas dans la liste des métiers à ne plus considérer), par ordre d'intervenant dont le taux de remplissage de Q va du plus faible au plus important, on cherche à déterminer un élève ayant un choix (de rang de préférence le plus élevé) correspondant à la catégorie de métier représenté par cet intervenant. 

Cet élève ne doit pas avoir visité cet intervenant auparavant.

Si un élève est trouvé, on retire l'élève de la liste des candidats et on ajoute cet élève à la Q de l'intervenant.

Si aucun élève n'est trouvé, on ajoute le métier représenté par l'intervenant à la liste des métiers à ne plus considérer (blacklist).

## Queing intervenants load balance driven

In [21]:
def q_students_itv_driven_pass1(n_slot:int, q_intervenants:Q_Intervenants, students:list[Student])->Tuple[Q_Intervenants, list[Student], list[Student]]:
    NB_LOOP_MAX = 1000

    result_students = get_copy_students(students)
    remaining_students = result_students
    
    no_more_Q = False
    blacklisted_jobs=[]
    nb_loops = 0

    while((len(remaining_students)>0) and (not no_more_Q)):

        assert(nb_loops < NB_LOOP_MAX)
        nb_loops += 1

        # liste des intervenants ayant une file de visite Q qui n'est pas encore remplie et
        # classée du taux de remplissage de Q le plus faible vers le plus élevé.
        id_intervenants = [ id_itv for id_itv in get_next_q_to_load_sorted(q_intervenants) ]

        # liste [(job, intervenant)] pour chaque intervenant identifié précédement
        l_job_itv = [ get_job_itv_from_itv_id(q_intervenants, id_intervenant) for id_intervenant in id_intervenants  ]

        # list [(job, intervenant)] pour les jobs qui ne sont pas blacklisté i.e
        # pour les jobs pour lesquels il existe encore des élèves qui souhaitent visiter ce job
        l_job_itv_no_blacklisted = [ (job,itv) for (job,itv) in l_job_itv if not job in blacklisted_jobs ]

        if (len(l_job_itv_no_blacklisted)==0):
            # Il n'y a plus de job a traiter. Soit toutes les Q d'intervenants sont remplies soit 
            # tous les jobs sont maintenant blacklistés (i.e. aucun élève trouvé ayant un souhait pour tous ces 
            # métiers)

            # exit while loop
            no_more_Q = True
        else:

            # le job avec l'intervenant ayant sa file d'attente la moins remplie
            # est en tête de liste avec :
            # head is (job,itv), tail is *_ unused
            (job, itv), *_ = l_job_itv_no_blacklisted
            
            # chercher l'élève qui a émis le souhait de rang le plus élevé pour 
            # le job identifié comme ayant un intervenant avec la file d'attente la moins
            # remplie
            #     
            most_interested_student = get_most_interested_student_for_job(remaining_students,job)
            if most_interested_student != None:

                # ajouter (job, itv) dans la liste des itv_visited (slots) de l'élève 
                most_interested_student['itv_visited'].append((n_slot, (job,itv)))

                # ajouter l'élève dans la Q de l'intervenant pour le job
                q_intervenants[job][itv]['Q'].append(most_interested_student['id'])

                # remove the most interested student from remaining students
                remaining_students = [ s for s in remaining_students if s['id'] != most_interested_student['id']]
            else:
                # aucun élève intéressé par le job en pass1, ajouter le job en blacklist
                blacklisted_jobs.append(job)

    return q_intervenants, result_students, remaining_students

In [22]:
def q_students_itv_driven_pass2(n_slot:int, q_intervenants:Q_Intervenants, students:list[Student], remaining_students:list[Student])->Tuple[Q_Intervenants, list[Student]]:
    result_students = get_copy_students(students)

    for student in remaining_students:
        
        # liste des intervenants ayant une file de visite Q qui n'est pas encore remplie et
        # classée du taux de remplissage de Q le plus faible vers le plus élevé.

        id_intervenants = [ id_itv for id_itv in get_next_q_to_load_sorted(q_intervenants) ]
        
        # liste [(job, intervenant)] pour chaque intervenant identifié précédement
        l_q_job_itv = [ get_job_itv_from_itv_id(q_intervenants, id_intervenant) for id_intervenant in id_intervenants  ]

        job_yet_visited = { j for (s,(j,_)) in student['itv_visited'] }
        job_itv_yet_visited = { (j,i) for (s,(j,i)) in student['itv_visited'] }

        # liste des (job, itv) pour lesquels job n'a pas encore été visité par l'élève
        next_q_job_itv_1 = [ (j,i) for (j,i) in l_q_job_itv if not j in job_yet_visited ]

        # liste des (job, itv) pour lesquels (job,itv) n'a pas encore été visité
        next_q_job_itv_2 = [ (j,i) for (j,i) in l_q_job_itv if not (j,i) in job_itv_yet_visited ]

        job, itv = None, None

        if (len(next_q_job_itv_1)==0):
            if (len(next_q_job_itv_2)==0):
                print("(!) free student")
            else:
                # solution par défaut: on refait visiter un même job mais pas le même intervenant
                job, itv = next_q_job_itv_2[0]
        else:
            # solution préférentielle : on fait visiter un nouveau job
            job, itv = next_q_job_itv_1[0]

        if job != None:
            r_student = [ s for s in result_students if s['id'] == student['id'] ][0]
            r_student['itv_visited'].append((n_slot, (job, itv)))
            
            q_intervenants[job][itv]['Q'].append(student['id'])


    return q_intervenants, result_students



In [90]:
def get_timetable_itv_driven(nb_slots:int, students:list[Student], intervenants_data:pd.DataFrame)->Tuple[Timetable, list[Student]]:
    timetable = []
    for n_slot in range(nb_slots):
        q_intervenants = init_q_intervenants(intervenants_data)
        q_intervenants, students, remaining_students = q_students_itv_driven_pass1(n_slot, q_intervenants, students)
        q_intervenants, students = q_students_itv_driven_pass2(n_slot, q_intervenants, students, remaining_students)
        timetable.append(q_intervenants)
    return timetable, students

## Tests

### Test 1 : q_students_itv_driven - allocation des élèves pour un slot

In [24]:
# test : q_students_itv_driven
test_intervenants_data = pd.DataFrame([
  [],
  [ { "id" : 10, "batch_size":3 }, { "id" : 11, "batch_size":2 } ],
  [ { "id" : 20, "batch_size":3 }, { "id" : 21, "batch_size":2 } ],
  [ { "id" : 30, "batch_size":2 } ],
  [ { "id" : 40, "batch_size":1 } ],
  [ { "id" : 50, "batch_size":1 } ]
])

test_students = [ 
    { "id" : 3100, "wishes" : [ 2,1,3 ], 'itv_visited' : [] }, 
    { "id" : 3101, "wishes" : [ 2,1,3 ], 'itv_visited' : []  }, 
    { "id" : 3102, "wishes" : [ 2,1,4 ], 'itv_visited' : []  }, 
    { "id" : 3103, "wishes" : [ 2,1,4 ], 'itv_visited' : []  },
    { "id" : 3104, "wishes" : [ 2,1,4 ], 'itv_visited' : []  }, 
    { "id" : 3105, "wishes" : [ 2,1,4 ], 'itv_visited' : []  }, 
    { "id" : 3106, "wishes" : [ 2,1,3 ], 'itv_visited' : []  }, 
    { "id" : 3107, "wishes" : [ 1,2,3 ], 'itv_visited' : []  },
    { "id" : 3108, "wishes" : [ 1,2,3 ], 'itv_visited' : []  }, 
    { "id" : 3109, "wishes" : [ 1,2,3 ], 'itv_visited' : []  }, 
    { "id" : 3110, "wishes" : [ 1,2,3 ], 'itv_visited' : []  }, 
    { "id" : 3111, "wishes" : [ 1,2,4 ], 'itv_visited' : []  },
    { "id" : 3112, "wishes" : [ 1,3,4 ], 'itv_visited' : []  }
]

test_q_intervenants = init_q_intervenants(test_intervenants_data)
test_q_intervenants, test_students, remaining_students = q_students_itv_driven_pass1(0, test_q_intervenants, test_students)
test_q_intervenants, test_students = q_students_itv_driven_pass2(0, test_q_intervenants, test_students, remaining_students)

#
# Expected result :
#

expected_result = ([[],
  [{'id': 10, 'batch_size': 3, 'Q': [3107, 3109, 3111]},
   {'id': 11, 'batch_size': 2, 'Q': [3108, 3110]}],
  [{'id': 20, 'batch_size': 3, 'Q': [3100, 3103, 3105]},
   {'id': 21, 'batch_size': 2, 'Q': [3101, 3104]}],
  [{'id': 30, 'batch_size': 2, 'Q': [3112, 3106]}],
  [{'id': 40, 'batch_size': 1, 'Q': [3102]}],
  [{'id': 50, 'batch_size': 1, 'Q': []}]],
 [{'id': 3100, 'wishes': [2, 1, 3], 'itv_visited': [(0, (2, 0))]},
  {'id': 3101, 'wishes': [2, 1, 3], 'itv_visited': [(0, (2, 1))]},
  {'id': 3102, 'wishes': [2, 1, 4], 'itv_visited': [(0, (4, 0))]},
  {'id': 3103, 'wishes': [2, 1, 4], 'itv_visited': [(0, (2, 0))]},
  {'id': 3104, 'wishes': [2, 1, 4], 'itv_visited': [(0, (2, 1))]},
  {'id': 3105, 'wishes': [2, 1, 4], 'itv_visited': [(0, (2, 0))]},
  {'id': 3106, 'wishes': [2, 1, 3], 'itv_visited': [(0, (3, 0))]},
  {'id': 3107, 'wishes': [1, 2, 3], 'itv_visited': [(0, (1, 0))]},
  {'id': 3108, 'wishes': [1, 2, 3], 'itv_visited': [(0, (1, 1))]},
  {'id': 3109, 'wishes': [1, 2, 3], 'itv_visited': [(0, (1, 0))]},
  {'id': 3110, 'wishes': [1, 2, 3], 'itv_visited': [(0, (1, 1))]},
  {'id': 3111, 'wishes': [1, 2, 4], 'itv_visited': [(0, (1, 0))]},
  {'id': 3112, 'wishes': [1, 3, 4], 'itv_visited': [(0, (3, 0))]}])

(test_q_intervenants, test_students) == expected_result

True

In [25]:
# test : get_timetable_itv_drivent

test_intervenants_data = pd.DataFrame([
  [],
  [ { "id" : 10, "batch_size":3 }, { "id" : 11, "batch_size":2 } ],
  [ { "id" : 20, "batch_size":3 }, { "id" : 21, "batch_size":2 } ],
  [ { "id" : 30, "batch_size":2 } ],
  [ { "id" : 40, "batch_size":1 } ],
  [ { "id" : 50, "batch_size":1 } ]
])

test_students = [ 
    { "id" : 3100, "wishes" : [ 2,1,3 ], 'itv_visited' : [] }, 
    { "id" : 3101, "wishes" : [ 2,1,3 ], 'itv_visited' : []  }, 
    { "id" : 3102, "wishes" : [ 2,1,4 ], 'itv_visited' : []  }, 
    { "id" : 3103, "wishes" : [ 2,1,4 ], 'itv_visited' : []  },
    { "id" : 3104, "wishes" : [ 2,1,4 ], 'itv_visited' : []  }, 
    { "id" : 3105, "wishes" : [ 2,1,4 ], 'itv_visited' : []  }, 
    { "id" : 3106, "wishes" : [ 2,1,3 ], 'itv_visited' : []  }, 
    { "id" : 3107, "wishes" : [ 1,2,3 ], 'itv_visited' : []  },
    { "id" : 3108, "wishes" : [ 1,2,3 ], 'itv_visited' : []  }, 
    { "id" : 3109, "wishes" : [ 1,2,3 ], 'itv_visited' : []  }, 
    { "id" : 3110, "wishes" : [ 1,2,3 ], 'itv_visited' : []  }, 
    { "id" : 3111, "wishes" : [ 1,2,4 ], 'itv_visited' : []  },
    { "id" : 3112, "wishes" : [ 1,3,4 ], 'itv_visited' : []  }
]


NB_SLOTS = 3
_, test_students = get_timetable_itv_driven(NB_SLOTS, test_students, test_intervenants_data)
timetable = convert_to_student_timetable(NB_SLOTS, test_students)
timetable


[[(3100, (0, (2, 0))),
  (3101, (0, (2, 1))),
  (3102, (0, (4, 0))),
  (3103, (0, (2, 0))),
  (3104, (0, (2, 1))),
  (3105, (0, (2, 0))),
  (3106, (0, (3, 0))),
  (3107, (0, (1, 0))),
  (3108, (0, (1, 1))),
  (3109, (0, (1, 0))),
  (3110, (0, (1, 1))),
  (3111, (0, (1, 0))),
  (3112, (0, (3, 0)))],
 [(3100, (1, (1, 1))),
  (3101, (1, (3, 0))),
  (3102, (1, (2, 0))),
  (3103, (1, (4, 0))),
  (3104, (1, (1, 0))),
  (3105, (1, (1, 1))),
  (3106, (1, (2, 1))),
  (3107, (1, (2, 0))),
  (3108, (1, (2, 1))),
  (3109, (1, (3, 0))),
  (3110, (1, (2, 0))),
  (3111, (1, (5, 0))),
  (3112, (1, (1, 0)))],
 [(3100, (2, (3, 0))),
  (3101, (2, (1, 0))),
  (3102, (2, (1, 1))),
  (3103, (2, (1, 0))),
  (3104, (2, (4, 0))),
  (3105, (2, (5, 0))),
  (3106, (2, (1, 1))),
  (3107, (2, (3, 0))),
  (3108, (2, (2, 0))),
  (3109, (2, (2, 0))),
  (3110, (2, (2, 1))),
  (3111, (2, (2, 1))),
  (3112, (2, (2, 0)))]]

In [26]:
test_q_intervenants = init_q_intervenants(test_intervenants_data)
get_timetable_intervenants(timetable, test_q_intervenants)

[{10: 3, 11: 2, 20: 3, 21: 2, 30: 2, 40: 1, 50: 0},
 {10: 2, 11: 2, 20: 3, 21: 2, 30: 2, 40: 1, 50: 1},
 {10: 2, 11: 2, 20: 3, 21: 2, 30: 2, 40: 1, 50: 1}]

In [27]:
get_evaluation(timetable, test_students, test_q_intervenants)

mode : itv-driven
overall_student_satisfaction : 0.872
overall ratio visited        : 1.0
nb_no_visits_itv             : 1


In [28]:
timetable_student_view(timetable)

defaultdict(<class 'list'>, {3100: ['2-0', '1-1', '3-0'], 3101: ['2-1', '3-0', '1-0'], 3102: ['4-0', '2-0', '1-1'], 3103: ['2-0', '4-0', '1-0'], 3104: ['2-1', '1-0', '4-0'], 3105: ['2-0', '1-1', '5-0'], 3106: ['3-0', '2-1', '1-1'], 3107: ['1-0', '2-0', '3-0'], 3108: ['1-1', '2-1', '2-0'], 3109: ['1-0', '3-0', '2-0'], 3110: ['1-1', '2-0', '2-1'], 3111: ['1-0', '5-0', '2-1'], 3112: ['3-0', '1-0', '2-0']})


defaultdict(list,
            {3100: ['2-0', '1-1', '3-0'],
             3101: ['2-1', '3-0', '1-0'],
             3102: ['4-0', '2-0', '1-1'],
             3103: ['2-0', '4-0', '1-0'],
             3104: ['2-1', '1-0', '4-0'],
             3105: ['2-0', '1-1', '5-0'],
             3106: ['3-0', '2-1', '1-1'],
             3107: ['1-0', '2-0', '3-0'],
             3108: ['1-1', '2-1', '2-0'],
             3109: ['1-0', '3-0', '2-0'],
             3110: ['1-1', '2-0', '2-1'],
             3111: ['1-0', '5-0', '2-1'],
             3112: ['3-0', '1-0', '2-0']})

## Application aux données réelles
### Intervenants driven
#### SQL

In [135]:
import sqlite3


In [201]:

con = sqlite3.connect("./data/2026/db-forum-notebook-20-itv-driven-2026-test.db")

In [235]:
#
# get students data
#

student_data = []
cur = con.cursor()
for row in cur.execute("SELECT * FROM Student"):
    student_data.append({ "id" : row[0], "wishes" : [int(row[2]), int(row[3]), int(row[4]), int(row[5]), int(row[6])], "itv_visited" : []})
pd.DataFrame(student_data)[0:5]

,id,wishes,itv_visited
0,3100,"[10, 15, 2, 13, 5]",[]
1,3101,"[12, 15, 19, 7, 11]",[]
2,3102,"[11, 12, 9, 6, 16]",[]
3,3103,"[9, 10, 6, 18, 5]",[]
4,3104,"[9, 10, 17, 2, 8]",[]


In [236]:
#
# intervenants data from SQL
#

cur = con.cursor()
intervenants_raw_data = []
for row in cur.execute("SELECT * FROM Intervenants"):
    intervenants_raw_data.append({ "id" : row[0], "metier" : row[4], "id_metier" : row[5], "batch_size":row[7]})
res = cur.execute("SELECT MAX(categori) FROM Intervenants")
max_metier = res.fetchone()[0]
intervenants_data_from_sql= []

for i in range(max_metier+1):
    intervenants_data_from_sql.append([ { "id" : e["id"], "batch_size" : e["batch_size"] } for e in intervenants_raw_data if e["metier"] == i])


intervenants_data_sql = pd.DataFrame(intervenants_data_from_sql)
intervenants_data_sql

,0,1,2,3,4,5,6
0,None,None,None,None,None,None,None
1,"{'id': 1, 'batch_size': 5}","{'id': 2, 'batch_size': 5}","{'id': 3, 'batch_size': 5}",None,None,None,None
2,"{'id': 4, 'batch_size': 5}","{'id': 5, 'batch_size': 5}",None,None,None,None,None
...,...,...,...,...,...,...,...
17,"{'id': 43, 'batch_size': 5}","{'id': 44, 'batch_size': 5}",None,None,None,None,None
18,"{'id': 45, 'batch_size': 5}",None,None,None,None,None,None
19,"{'id': 46, 'batch_size': 5}","{'id': 47, 'batch_size': 5}","{'id': 48, 'batch_size': 5}","{'id': 49, 'batch_size': 5}",None,None,None


In [237]:
NB_SLOTS = 6

timetable, student_data = get_timetable_itv_driven(NB_SLOTS, student_data, intervenants_data_sql)
timetable = convert_to_student_timetable(NB_SLOTS, student_data)
q_intervenants = init_q_intervenants(intervenants_data_sql)

pd.set_option("display.max_columns", 65)
pd.set_option("display.max_rows", 6)
pd.DataFrame(get_timetable_intervenants(timetable, q_intervenants))

,1,2,3,4,5,6,7,8,9,87,10,88,89,11,12,13,14,15,90,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,36,37,38,39,40,41,42,43,44,45,46,47,48,49
0,5,4,4,4,4,5,3,4,4,3,4,4,4,4,4,4,4,4,4,4,4,4,4,5,4,5,4,5,4,3,3,3,3,4,4,4,4,4,4,4,5,5,4,4,4,4,4,5,4,4,4,4
1,5,4,4,4,4,4,4,3,3,3,4,4,4,4,4,4,4,4,4,4,4,4,4,5,4,5,4,4,5,4,3,3,3,3,4,4,4,5,5,4,5,4,4,5,4,4,4,5,4,4,4,4
2,5,5,5,5,5,5,5,3,3,3,5,5,5,5,5,4,4,4,4,4,4,5,5,5,4,4,5,4,4,5,3,3,3,3,3,3,2,3,3,3,5,4,3,5,4,5,5,5,3,3,2,3
3,5,5,5,5,5,5,5,4,3,4,5,5,5,5,5,4,4,4,4,5,4,4,4,4,4,4,4,4,4,4,4,3,3,3,3,3,3,4,3,3,5,3,3,5,4,4,3,5,3,3,4,4
4,5,5,5,5,5,5,5,3,4,4,5,5,5,5,4,4,4,3,3,3,4,4,3,3,3,4,4,4,4,4,3,4,4,4,4,4,4,4,4,4,5,3,3,5,5,3,3,5,3,4,4,4
5,4,4,4,4,4,5,5,3,4,4,4,3,4,5,4,4,4,4,4,4,4,3,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,5,4,4,4,4,4,4,5,4,4,4,4


In [238]:
get_evaluation(timetable, student_data, q_intervenants)

mode : itv-driven
overall_student_satisfaction : 0.924
overall ratio visited        : 1.2
nb_no_visits_itv             : 0


In [246]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 240)
timetable_students_sql = pd.DataFrame(timetable_student_view(timetable)).T
timetable_students_sql[0:5]

defaultdict(<class 'list'>, {3100: ['10-1', '15-1', '2-1', '13-2', '5-2', '11-0'], 3101: ['12-6', '15-0', '19-3', '11-1', '10-2', '13-1'], 3102: ['11-1', '12-5', '9-1', '6-1', '16-1', '10-1'], 3103: ['9-4', '10-1', '5-2', '6-0', '19-3', '18-0'], 3104: ['9-2', '10-0', '17-1', '8-1', '2-0', '19-3'], 3105: ['1-0', '19-1', '3-0', '6-2', '2-0', '8-4'], 3106: ['8-4', '5-0', '6-1', '1-2', '2-1', '9-1'], 3107: ['19-3', '12-3', '8-2', '16-1', '10-0', '9-0'], 3108: ['7-0', '18-0', '17-0', '10-0', '8-4', '12-1'], 3109: ['6-2', '1-0', '19-1', '5-2', '2-0', '8-2'], 3110: ['19-1', '10-2', '17-1', '15-1', '12-5', '18-0'], 3111: ['8-3', '10-3', '12-0', '19-2', '7-0', '14-0'], 3112: ['10-0', '17-0', '7-0', '18-0', '19-1', '8-1'], 3113: ['17-1', '10-1', '9-4', '8-4', '1-0', '13-0'], 3114: ['17-0', '19-3', '8-4', '5-0', '6-2', '12-0'], 3115: ['6-1', '17-0', '9-4', '19-2', '8-3', '10-3'], 3116: ['8-1', '17-1', '10-3', '9-0', '19-0', '12-2'], 3117: ['8-0', '12-4', '9-2', '19-3', '15-1', '10-2'], 3118: ['6-

,0,1,2,3,4,5
3100,10-1,15-1,2-1,13-2,5-2,11-0
3101,12-6,15-0,19-3,11-1,10-2,13-1
3102,11-1,12-5,9-1,6-1,16-1,10-1
3103,9-4,10-1,5-2,6-0,19-3,18-0
3104,9-2,10-0,17-1,8-1,2-0,19-3


In [ ]:
timetable_students_sql.to_csv('output_timetable_students_itv_driven_2026_from_sql.csv', sep=';', header=False)

#### JSON

In [ ]:
NB_SLOTS = 6
FILE_INTERVENANTS_DATA = './data/2026/intervenants_data_2026.json'
FILE_STUDENTS = './data/2026/data_2026.json'


,0,1,2,3,4,5,6
0,None,None,None,None,None,None,None
1,"{'id': 10, 'batch_size': 5}","{'id': 11, 'batch_size': 5}","{'id': 12, 'batch_size': 5}",None,None,None,None
2,"{'id': 20, 'batch_size': 5}","{'id': 21, 'batch_size': 5}",None,None,None,None,None
...,...,...,...,...,...,...,...
17,"{'id': 170, 'batch_size': 5}","{'id': 171, 'batch_size': 5}",None,None,None,None,None
18,"{'id': 180, 'batch_size': 5}",None,None,None,None,None,None
19,"{'id': 190, 'batch_size': 5}","{'id': 191, 'batch_size': 5}","{'id': 192, 'batch_size': 5}","{'id': 193, 'batch_size': 5}",None,None,None


In [187]:

#
# student data from JSON
#

raw_students_data = pd.read_json(FILE_STUDENTS)
raw_students_data_json = raw_students_data.to_json(orient="index")
parsed = loads(raw_students_data_json)
student_data = list(parsed.values())
student_data[0:5]

[{'id': 3100, 'wishes': [10, 15, 2, 13, 5], 'itv_visited': []},
 {'id': 3101, 'wishes': [12, 15, 19, 7, 11], 'itv_visited': []},
 {'id': 3102, 'wishes': [11, 12, 9, 6, 16], 'itv_visited': []},
 {'id': 3103, 'wishes': [9, 10, 6, 18, 5], 'itv_visited': []},
 {'id': 3104, 'wishes': [9, 10, 17, 2, 8], 'itv_visited': []}]

In [ ]:

#
# intervenants data from JSON
#

intervenants_data_json = pd.read_json(FILE_INTERVENANTS_DATA)
intervenants_data_json


In [143]:
timetable, student_data = get_timetable_itv_driven(NB_SLOTS, student_data, intervenants_data_json)
timetable = convert_to_student_timetable(NB_SLOTS, student_data)
q_intervenants = init_q_intervenants(intervenants_data_json)

pd.set_option("display.max_columns", 65)
pd.set_option("display.max_rows", 6)
pd.DataFrame(get_timetable_intervenants(timetable, q_intervenants))

,10,11,12,20,21,30,40,50,51,52,60,61,62,70,80,81,82,83,84,90,91,92,93,94,100,101,102,103,110,111,120,121,122,123,124,125,126,130,131,132,140,150,151,160,161,170,171,180,190,191,192,193
0,5,4,4,4,4,5,3,4,4,3,4,4,4,4,4,4,4,4,4,4,4,4,4,5,4,5,4,5,4,3,3,3,3,4,4,4,4,4,4,4,5,5,4,4,4,4,4,5,4,4,4,4
1,5,4,4,4,4,4,4,3,3,3,4,4,4,4,4,4,4,4,4,4,4,4,4,5,4,5,4,4,5,4,3,3,3,3,4,4,4,5,5,4,5,4,4,5,4,4,4,5,4,4,4,4
2,5,5,5,5,5,5,5,3,3,3,5,5,5,5,5,4,4,4,4,4,4,5,5,5,4,4,5,4,4,5,3,3,3,3,3,3,2,3,3,3,5,4,3,5,4,5,5,5,3,3,2,3
3,5,5,5,5,5,5,5,4,3,4,5,5,5,5,5,4,4,4,4,5,4,4,4,4,4,4,4,4,4,4,4,3,3,3,3,3,3,4,3,3,5,3,3,5,4,4,3,5,3,3,4,4
4,5,5,5,5,5,5,5,3,4,4,5,5,5,5,4,4,4,3,3,3,4,4,3,3,3,4,4,4,4,4,3,4,4,4,4,4,4,4,4,4,5,3,3,5,5,3,3,5,3,4,4,4
5,4,4,4,4,4,5,5,3,4,4,4,3,4,5,4,4,4,4,4,4,4,3,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,5,4,4,4,4,4,4,5,4,4,4,4


In [118]:
get_evaluation(timetable, student_data, q_intervenants)

mode : itv-driven
overall_student_satisfaction : 0.924
overall ratio visited        : 1.2
nb_no_visits_itv             : 0


In [188]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 240)
timetable_students_json = pd.DataFrame(timetable_student_view(timetable)).T
timetable_students_json[0:5]

defaultdict(<class 'list'>, {3100: ['10-1', '15-1', '2-1', '13-2', '5-2', '11-0'], 3101: ['12-6', '15-0', '19-3', '11-1', '10-2', '13-1'], 3102: ['11-1', '12-5', '9-1', '6-1', '16-1', '10-1'], 3103: ['9-4', '10-1', '5-2', '6-0', '19-3', '18-0'], 3104: ['9-2', '10-0', '17-1', '8-1', '2-0', '19-3'], 3105: ['1-0', '19-1', '3-0', '6-2', '2-0', '8-4'], 3106: ['8-4', '5-0', '6-1', '1-2', '2-1', '9-1'], 3107: ['19-3', '12-3', '8-2', '16-1', '10-0', '9-0'], 3108: ['7-0', '18-0', '17-0', '10-0', '8-4', '12-1'], 3109: ['6-2', '1-0', '19-1', '5-2', '2-0', '8-2'], 3110: ['19-1', '10-2', '17-1', '15-1', '12-5', '18-0'], 3111: ['8-3', '10-3', '12-0', '19-2', '7-0', '14-0'], 3112: ['10-0', '17-0', '7-0', '18-0', '19-1', '8-1'], 3113: ['17-1', '10-1', '9-4', '8-4', '1-0', '13-0'], 3114: ['17-0', '19-3', '8-4', '5-0', '6-2', '12-0'], 3115: ['6-1', '17-0', '9-4', '19-2', '8-3', '10-3'], 3116: ['8-1', '17-1', '10-3', '9-0', '19-0', '12-2'], 3117: ['8-0', '12-4', '9-2', '19-3', '15-1', '10-2'], 3118: ['6-

,0,1,2,3,4,5
3100,10-1,15-1,2-1,13-2,5-2,11-0
3101,12-6,15-0,19-3,11-1,10-2,13-1
3102,11-1,12-5,9-1,6-1,16-1,10-1
3103,9-4,10-1,5-2,6-0,19-3,18-0
3104,9-2,10-0,17-1,8-1,2-0,19-3


In [ ]:
timetable_students_json.to_csv('output_timetable_students_itv_driven_2026_from_json.csv', sep=';', header=False)